<a href="https://colab.research.google.com/github/merazlab/pytorch/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [0]:
input_size = 784
hidden_size = 400
out_size = 10
epochs = 10
batch_size = 100
lr = 0.001

In [4]:
train_dataset = datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=True)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw





Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
train_loder = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loder = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, out_size):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, out_size)
    self.relu = nn.ReLU()
    self.init_weights()

  def init_weights(self):
    nn.init.kaiming_normal(self.fc1.weight)
    nn.init.kaiming_normal(self.fc2.weight)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out= self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    return out

In [11]:
net = Net(input_size, hidden_size, out_size)
CUDA = torch.cuda.is_available()
if CUDA:
  net = net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if sys.path[0] == '':


In [21]:
for epoch in range(epochs):
  correct_train = 0
  running_loss = 0
  for i, (images, labels) in enumerate(train_loder):
    images = images.view(-1, 28*28)
    if CUDA:
      images = images.cuda()
      labels = labels.cuda()

    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    correct_train += (predicted == labels).sum()
    loss = criterion(outputs, labels)

    running_loss +=loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch [{}/{}], Training Loss: {:.3f}, Training Accuracy: {:.3f}%'.format
    (epoch+1, epochs, running_loss/len(train_loder), (100*correct_train.double()/len(train_dataset))))
print("DONE TRAINING!")


Epoch [1/10], Training Loss: 0.016, Training Accuracy: 99.475%
Epoch [2/10], Training Loss: 0.013, Training Accuracy: 99.572%
Epoch [3/10], Training Loss: 0.015, Training Accuracy: 99.513%
Epoch [4/10], Training Loss: 0.011, Training Accuracy: 99.638%
Epoch [5/10], Training Loss: 0.008, Training Accuracy: 99.750%
Epoch [6/10], Training Loss: 0.010, Training Accuracy: 99.665%
Epoch [7/10], Training Loss: 0.012, Training Accuracy: 99.612%
Epoch [8/10], Training Loss: 0.009, Training Accuracy: 99.707%
Epoch [9/10], Training Loss: 0.009, Training Accuracy: 99.707%
Epoch [10/10], Training Loss: 0.009, Training Accuracy: 99.705%
DONE TRAINING!
